#### Setup Environment

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# Administering Your GIS Organizations Using ArcGIS API for Python

## Overview

<img src="./img/gettingstarted.jpg" width=700/>

- The ArcGIS ecosystem is vast
- Organizations can have multiple versions of any product or multiple products to manage
- How do you manage this?

## The Way of the Python

<img src="./img/waysofscience.jpg" width=700/>

- The Python API allows administrators to manage, update and control what happens on your server
- Script from your favorite IDE or Notebook environment
- Cross platform support

## What Can We Manage?

<table  style='font-family:"Courier New", Courier, monospace; font-size:200%' width=50%>
  
  <tr>
    <td>Users</td>
    <td><img src="./img/users.png", width=50/></td>
  </tr>
  <tr>
    <td>Content</td>
    <td><img src="./img/content.png", width=50/></td>
  </tr>
  <tr>
    <td>Organization Policies/Licensing</td>
    <td><img src="./img/infrastructure.png", width=50/></td>
  </tr>
  <tr>
    <td>Groups</td>
    <td><img src="./img/groups.png" width=50/></td>
  </tr>
 
</table>

## Getting Started

### Understand the `GIS` Object

The `GIS` object is the way users connect to ArcGIS Online and/or Enterprise

- It doesn't matter if you are an administrator of a user, we must start here.

#### Connecting to you `GIS` 

The ArcGIS API for Python support multiple ways of connecting to the `GIS`, which is ArcGIS Online or ArcGIS Enterprise

##### Anonymously

In [ ]:
import pandas as pd
from arcgis.gis import GIS
gis = GIS() #anonymous connection

In [ ]:
gis.content.search("rivers")

##### Built-In

- username/password login method
- usersname are case sensitive 

```python
gis = GIS(username='fakeaccount', password='fakepassword')
gis = GIS(url="https://www.mysite.com/portal", username='fakeaccount', password='fakepassword')
```

**Protecting Built-In Credentials**

- using `profiles` will help protect username and passwords.  
- prevents accidental sharing

1. Create a `GIS` object with the extra `profile` parameter

```python
gis = GIS(url="https://www.mysite.com/portal", 
          username='fakeaccount', 
          password='fakepassword', 
          profile='portal_profile')
```

2. Now connect using the `profile`

```python
gis = GIS(profile='portal_profile')
```

**What Happened?**

Instead of keeping your password in plain text, now we leverage the operating system's credential store for the logged in user.  The credentials never get passed on when you use profiles.

##### Other Login Methods

- LDAP
- IWA
- PKI
- OAuth 2.0.
- API Key

## User Management

<img src="./img/usermanagement.jpg"/>


Users fuel your system. As an administrator your job is to ensure they can put up there content and know the site is reliable and safe.  The Python API is a tool to do just that!

In [ ]:
from arcgis.gis import GIS, UserManager

In [ ]:
gis = GIS(profile='your_online_profile', verify_cert=False)

# Working with Existing Users

In [ ]:
um:UserManager = gis.users
um

#### Search for Users

In [ ]:
users = um.search("and*")
users

#### List User's Groups

In [ ]:
um.user_groups(um.search("and*"))

#### Access a User's Items

In [ ]:
users[1].items()

In [ ]:
user = um.search("geosaurus")[0]
user.update(
    thumbnail=r"./img/dino.png"
)
user

### Creating New Users

In [ ]:
# Check the user types available
um.counts('user_type')

In [ ]:
import uuid
username = f"RUser{uuid.uuid4().hex[:4]}"
password = f"!{uuid.uuid4().hex[:8]}A"

In [ ]:
new_user = um.create(username=username, password=password, 
                     firstname="Danny", lastname="Human", 
                     email='testsadf@esri.com')
new_user

In [ ]:
um.counts('user_type')

### Handling Security and Password Resets

<img src="./imgs/password-reset.jpg"/>

In [ ]:
new_password = f"!{uuid.uuid4().hex[:8]}A"

In [ ]:
new_user.reset(
    password=password,
    new_password=new_password,
    new_security_question=1,
    new_security_answer=uuid.uuid4().hex[:10],
    reset_by_email=False,
)

In [ ]:
GIS(username=username, 
    password=new_password, 
    verify_cert=False).users.me

### Deleting the User

- user must not own items
- user must not have licenses checked out

In [ ]:
new_user.delete()

### Working with Roles and User Types

<img src="./img/know_your_role.jpg" width=500/>

#### User types

- User type determines the privileges that can be granted to the member through a default or custom role
- Common Roles:
  + viewer, creator and administrator

In [ ]:
gis = GIS(profile='your_online_profile')

In [ ]:
user = gis.users.search("uc2023*")[0]

In [ ]:
user.user_types()['id']

In [ ]:
user.update_license_type("GISProfessionalAdvUT")
user.user_types()['id']

In [ ]:
user.update_license_type("creatorUT")

In [ ]:
user.user_types()['id']

#### Working with Roles

- A role defines the set of privileges assigned to a member

**Accessing Role Manager**

In [ ]:
rm = gis.users.roles
rm

**Listing Roles**

- notice that roles can have the same name!


In [ ]:
rm.all()

**Check for Existence of a Role**

In [ ]:
rm.exists('DataEditorRole')

In [ ]:
role = rm.create(name="DataEditorRole", 
                 description="Allow to modify service data", 
                 privileges=[
                        "features:user:edit",
                        "features:user:fullEdit",
                        "opendata:user:designateGroup",
                        "portal:admin:viewUsers",
                        "portal:user:createGroup"]
                )
role

In [ ]:
username = f"RUser{uuid.uuid4().hex[:4]}"
password = f"!{uuid.uuid4().hex[:8]}A"
new_user = um.create(username=username, password=password, 
                     firstname="Danny", lastname="Human", 
                     email='testsadf@esri.com')
new_user

In [ ]:
new_user.update_role(role)

In [ ]:
new_user.role, role.role_id

**Removing the Role**

In [ ]:
new_user.delete()
role.delete()